# 🏗️ Floorplan SDXL LoRA Training Notebook  
A clean, organized workflow for training, validating, saving, and exporting LoRA models.


# 📦 Step 1: Install & Import Dependencies

In this step, we install all required libraries and import essential modules for
training, validation, and inference.  
We ensure GPU is enabled and prepare the environment for Stable Diffusion XL training.


In [ ]:
!pip uninstall -y diffusers accelerate transformers peft

!pip install diffusers==0.29.2
!pip install transformers==4.41.2
!pip install accelerate==0.27.2
!pip install peft==0.10.0
!pip install bitsandbytes safetensors einops


Found existing installation: diffusers 0.29.2
Uninstalling diffusers-0.29.2:
  Successfully uninstalled diffusers-0.29.2
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: peft 0.10.0
Uninstalling peft-0.10.0:
  Successfully uninstalled peft-0.10.0
  Using cached diffusers-0.29.2-py3-none-any.whl.metadata (19 kB)
Using cached diffusers-0.29.2-py3-none-any.whl (2.2 MB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
  Using cached accelerate-0.27.2-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)
  Using cached peft-0.10.0-py3-none-any.whl.metadata (13 kB)
Using cached peft-0.10.0-py3-none-any.whl (199 kB)


In [ ]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/v0.29.2/examples/text_to_image/train_text_to_image_lora_sdxl.py


--2025-11-27 20:23:16--  https://raw.githubusercontent.com/huggingface/diffusers/v0.29.2/examples/text_to_image/train_text_to_image_lora_sdxl.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55538 (54K) [text/plain]
Saving to: ‘train_text_to_image_lora_sdxl.py’

train_text_to_image 100%[===================>]  54.24K  --.-KB/s    in 0.02s   

2025-11-27 20:23:16 (2.56 MB/s) - ‘train_text_to_image_lora_sdxl.py’ saved [55538/55538]



In [ ]:
!pip install diffusers==0.29.2 transformers accelerate peft==0.10.0 bitsandbytes safetensors datasets


In [ ]:
import diffusers, accelerate, peft
print("DIFFUSERS:", diffusers.__version__)
print("ACCELERATE:", accelerate.__version__)
print("PEFT:", peft.__version__)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

DIFFUSERS: 0.29.2
ACCELERATE: 0.27.2
PEFT: 0.10.0


# 🗂️ Step 2: Load & Inspect Dataset

We load the dataset and verify that the CSV and image files are properly aligned.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#/content/drive/MyDrive/final_dataset.zip
!unzip "/content/drive/MyDrive/final_dataset.zip" -d "/content/final_dataset"


Archive:  /content/drive/MyDrive/final_dataset.zip
  inflating: /content/final_dataset/resized_dataset.csv  
  inflating: /content/final_dataset/resized_images/image_0.png  
  inflating: /content/final_dataset/resized_images/image_1.png  
  inflating: /content/final_dataset/resized_images/image_10.png  
  inflating: /content/final_dataset/resized_images/image_10001.png  
  inflating: /content/final_dataset/resized_images/image_10002.png  
  inflating: /content/final_dataset/resized_images/image_10003.png  
  inflating: /content/final_dataset/resized_images/image_10004.png  
  inflating: /content/final_dataset/resized_images/image_10005.png  
  inflating: /content/final_dataset/resized_images/image_10006.png  
  inflating: /content/final_dataset/resized_images/image_10007.png  
  inflating: /content/final_dataset/resized_images/image_10008.png  
  inflating: /content/final_dataset/resized_images/image_10009.png  
  inflating: /content/final_dataset/resized_images/image_10010.png  
  inf

In [16]:
import pandas as pd
import re

INPUT_CSV = "/content/final_dataset/resized_dataset.csv"
df = pd.read_csv(INPUT_CSV)
df.head()

,id,Text,image
0,0,bedrooms : 2 | toilet : 2 | attached _ toilet ...,resized_images\image_0.png
1,1,bedrooms : 2 | toilet : 2 | attached _ toilet ...,resized_images\image_1.png
2,2,bedrooms : 3 | toilet : 2 | attached _ toilet ...,resized_images\image_2.png
3,3,bedrooms : 1 | toilet : 2 | attached _ toilet ...,resized_images\image_3.png
4,5,bedrooms : 1 | toilet : 1 | attached _ toilet ...,resized_images\image_5.png


# 🧹 Step 3 — Text Preprocessing

Before training the model, we clean and standardize all text captions.  
This makes the dataset more consistent and helps the model learn more efficiently.

## 🔧 What happens in this step:

- Remove extra spaces  
- Convert all text to lowercase  
- Fix common formatting issues  
- Ensure each caption matches the correct image  


After preprocessing, the captions become cleaner, more uniform, and better suited for training.


In [4]:
import pandas as pd
import re

INPUT_CSV = "/content/final_dataset/resized_dataset.csv"
OUTPUT_CSV = "/content/final_clean_dataset.csv"

df = pd.read_csv(INPUT_CSV)

def clean_text_keep_everything(raw):
    if not isinstance(raw, str):
        return ""

    text = raw

    # remove weird unicode characters
    text = text.encode('ascii', 'ignore').decode()

    # normalize underscores
    text = text.replace("_", " ")

    # add spaces around separators
    text = re.sub(r"\|", " | ", text)

    # normalize parentheses spacing
    text = re.sub(r"\(", " (", text)
    text = re.sub(r"\)", ") ", text)

    # collapse multiple spaces
    text = re.sub(r"\s+", " ", text)

    text = text.replace("|", ",")

    # strip leading and trailing spaces
    text = text.strip()

    # ensure ending period
    if not text.endswith("."):
        text += "."

    return text


# Create final caption for each row
df["text"] = df["Text"].apply(clean_text_keep_everything)

# Keep only required columns
df = df[["image", "text"]]

df.to_csv(OUTPUT_CSV, index=False)
print("✔ Final cleaned CSV saved to:", OUTPUT_CSV)

df.head()

✔ Final cleaned CSV saved to: /content/final_clean_dataset.csv


,image,text
0,resized_images\image_0.png,"bedrooms : 2 , toilet : 2 , attached toilet : ..."
1,resized_images\image_1.png,"bedrooms : 2 , toilet : 2 , attached toilet : ..."
2,resized_images\image_2.png,"bedrooms : 3 , toilet : 2 , attached toilet : ..."
3,resized_images\image_3.png,"bedrooms : 1 , toilet : 2 , attached toilet : ..."
4,resized_images\image_5.png,"bedrooms : 1 , toilet : 1 , attached toilet : ..."


In [3]:
DATA_DIR = "/content/final_dataset/resized_images"
CAPTION_CSV = "/content/final_clean_dataset.csv"
OUTPUT_DIR = "/content/drive/MyDrive/floorplan-lora"

In [5]:
import pandas as pd
import re

df = pd.read_csv("/content/final_clean_dataset.csv")

df["image"] = df["image"].astype(str)

df["text"] = df["text"].astype(str)

df.head()
df.dtypes


,0
image,object
text,object


In [6]:
import pandas as pd
import os

CSV_PATH = "/content/final_clean_dataset.csv"
IMAGE_DIR = "/content/final_dataset/resized_images"

df = pd.read_csv(CSV_PATH)

for _, row in df.iterrows():
    image_path = row["image"].replace("\\", "/")
    caption = row["text"]

    name = os.path.basename(image_path)
    base = os.path.splitext(name)[0]

    with open(os.path.join(IMAGE_DIR, base + ".txt"), "w") as f:
        f.write(caption)

print("✔️ Captions created!")

✔️ Captions created!


In [7]:
from datasets import Dataset
train_data = Dataset.from_pandas(df)
train_data


Dataset({
    features: ['image', 'text'],
    num_rows: 1605
})

In [8]:
import pandas as pd, shutil, os

df = pd.read_csv("/content/final_clean_dataset.csv")

# 1. Fix Windows backslashes
df["image"] = df["image"].str.replace("\\", "/", regex=False)

# 2. Add the correct folder prefix where your images really are
df["image"] = "/content/final_dataset/" + df["image"]

In [9]:
os.makedirs("train_data", exist_ok=True)

missing = 0

for i, row in df.iterrows():
    src = row["image"]
    new_name = f"{i}.png"
    dst = f"train_data/{new_name}"

    if os.path.exists(src):
        shutil.copy(src, dst)
        df.loc[i, "image"] = new_name
    else:
        print("Missing:", src)
        missing += 1

df[["image","text"]].to_csv("train_data/metadata.csv", index=False)

print("DONE! Missing files:", missing)

DONE! Missing files: 0


In [10]:
import pandas as pd
import shutil, os

df = pd.read_csv("/content/final_clean_dataset.csv")

# Fix slashes AND point to the real folder
df["image"] = "/content/final_dataset/" + df["image"].str.replace("\\", "/", regex=False)

os.makedirs("images", exist_ok=True)

missing = 0
for path in df["image"]:
    if os.path.exists(path):
        shutil.copy(path, "images/")
    else:
        print("Missing:", path)
        missing += 1

print("Total missing:", missing)

df.to_csv("metadata.csv", index=False)

Total missing: 0


In [12]:
import pandas as pd, shutil, os

df = pd.read_csv("/content/final_clean_dataset.csv")

os.makedirs("final_train", exist_ok=True)

# List all real images in train_data
real_images = sorted([f for f in os.listdir("train_data") if f.lower().endswith((".png",".jpg",".jpeg"))])

if len(real_images) != len(df):
    print("WARNING: Image count and CSV rows do NOT match!")
    print("CSV rows:", len(df))
    print("Images found:", len(real_images))

# Rename all images 0.png, 1.png, 2.png...
for i, real_image in enumerate(real_images):
    new_name = f"{i}.png"
    shutil.copy(f"train_data/{real_image}", f"final_train/{new_name}")
    df.loc[i, "image"] = new_name

# Save metadata
df[["image","text"]].to_csv("final_train/metadata.csv", index=False)

print("DONE! Images renamed + metadata created.")

DONE! Images renamed + metadata created.


In [13]:
import pandas as pd
pd.read_csv("train_data/metadata.csv").head()


,image,text
0,0.png,"bedrooms : 2 , toilet : 2 , attached toilet : ..."
1,1.png,"bedrooms : 2 , toilet : 2 , attached toilet : ..."
2,2.png,"bedrooms : 3 , toilet : 2 , attached toilet : ..."
3,3.png,"bedrooms : 1 , toilet : 2 , attached toilet : ..."
4,4.png,"bedrooms : 1 , toilet : 1 , attached toilet : ..."


In [14]:
import pandas as pd

df = pd.read_csv("train_data/metadata.csv")
df.rename(columns={"image": "file_name"}, inplace=True)
df.to_csv("train_data/metadata.csv", index=False)

print(df.head())


  file_name                                               text
0     0.png  bedrooms : 2 , toilet : 2 , attached toilet : ...
1     1.png  bedrooms : 2 , toilet : 2 , attached toilet : ...
2     2.png  bedrooms : 3 , toilet : 2 , attached toilet : ...
3     3.png  bedrooms : 1 , toilet : 2 , attached toilet : ...
4     4.png  bedrooms : 1 , toilet : 1 , attached toilet : ...


In [15]:
from datasets import Dataset
train_data = Dataset.from_pandas(df)
train_data


Dataset({
    features: ['file_name', 'text'],
    num_rows: 1605
})

# 🚀 Step 4: Start Training LoRA

This cell launches the actual LoRA training using Accelerate.
During training:
- Checkpoints are saved every 500 steps  
- Step loss is printed  
- GPU utilization is monitored  

If the notebook disconnects, you can resume training from the latest checkpoint.


In [ ]:
!accelerate launch train_text_to_image_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --train_data_dir="train_data" \
  --caption_column="text" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_train_epochs=6  \
  --output_dir="floorplan_lora" \
  --checkpointing_steps=500 \
  --output_dir="/content/drive/MyDrive/floorplan_lora" \
  --mixed_precision="fp16" \
  --use_8bit_adam


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-27 13:01:05.218201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764248465.237819    1829 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764248465.243735    1829 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764248465.258651    1829 computation_plac

# 🔍 Step 5: Validate LoRA Checkpoints

Here we load each checkpoint (500, 1000, 1500, 2000)  
and generate one test image.  
This helps determine which checkpoint gives the best quality output.


In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16
).to("cuda")

pipe.load_lora_weights("/content/floorplan_lora/pytorch_lora_weights.safetensors")

prompt = "bedrooms : 5 , toilet : 3 , attached toilet : 3 , dress room : 0 , features :living room, dining, kitchen"

image = pipe(
    prompt,
    num_inference_steps=60,   # Higher = better quality
    guidance_scale=7.5          # Controls prompt strength
).images[0]

image.save("test.png")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vae_1_0/diffusion_pytorch_model.safetens(…):   0%|          | 0.00/335M [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

# 🏅 Step 6: Select the Best Checkpoint

After testing all checkpoints, choose the best performing one based on:
- Image clarity  
- Room count accuracy  
- Door/window accuracy  
- Line consistency  


In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch
import os

# ----------------------
# PATHS
# ----------------------
BASE_MODEL = "stabilityai/stable-diffusion-xl-base-1.0"
LORA_DIR = "/content/floorplan_lora"

CHECKPOINTS = {
    "final": f"{LORA_DIR}/pytorch_lora_weights.safetensors",
    "500": f"{LORA_DIR}/checkpoint-500/pytorch_lora_weights.safetensors",
    "1000": f"{LORA_DIR}/checkpoint-1000/pytorch_lora_weights.safetensors",
    "1500": f"{LORA_DIR}/checkpoint-1500/pytorch_lora_weights.safetensors",
    "2000": f"{LORA_DIR}/checkpoint-2000/pytorch_lora_weights.safetensors",
}

SAVE_DIR = "/content/lora_comparison"
os.makedirs(SAVE_DIR, exist_ok=True)

# ----------------------
# Load Base Pipeline
# ----------------------
pipe = StableDiffusionXLPipeline.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
).to("cuda")

prompt = "bedrooms : 3 , toilet : 2 , attached toilet : 1 , dress room : 0 , features :drawing room, dining, kitchen, rear lawn."

# ----------------------
# Generate images for each checkpoint
# ----------------------
for name, ckpt_path in CHECKPOINTS.items():
    print(f"\n🔥 Generating for checkpoint: {name}")

    # Load LoRA
    pipe.unload_lora_weights()  # Clear previous LoRA
    pipe.load_lora_weights(ckpt_path)

    # Generate image
    image = pipe(
        prompt,
        num_inference_steps=50,
        guidance_scale=7.5
    ).images[0]

    save_path = f"{SAVE_DIR}/{name}.png"
    image.save(save_path)
    print(f"✔️ Saved: {save_path}")

print("\n🎉 DONE! All comparison images saved in /content/lora_comparison")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!



🔥 Generating for checkpoint: final


No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder_2'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


...

  0%|          | 0/50 [00:00<?, ?it/s]

✔️ Saved: /content/lora_comparison/1500.png

🔥 Generating for checkpoint: 2000


No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder_2'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


  0%|          | 0/50 [00:00<?, ?it/s]

✔️ Saved: /content/lora_comparison/2000.png

🎉 DONE! All comparison images saved in /content/lora_comparison


# 🌐 Step 7: Build a Flask API

we deploy a Flask API that receives JSON requests
and returns generated floorplan images.  
Flutter can call the API instead of running SDXL locally.


In [22]:
!pip install diffusers transformers accelerate safetensors --upgrade



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 95.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2


In [23]:
import os

path = "/content/drive/MyDrive/floorplan_lora"
print(os.listdir(path))
# before resume

['checkpoint-500', 'checkpoint-1000', 'checkpoint-1500', 'checkpoint-2000', 'pytorch_lora_weights.safetensors']


In [24]:
!pip uninstall -y diffusers accelerate transformers peft

!pip install diffusers==0.29.2
!pip install transformers==4.41.2
!pip install accelerate==0.27.2
!pip install peft==0.10.0
!pip install bitsandbytes safetensors einops

Found existing installation: diffusers 0.35.2
Uninstalling diffusers-0.35.2:
  Successfully uninstalled diffusers-0.35.2
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 114.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [25]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/v0.29.2/examples/text_to_image/train_text_to_image_lora_sdxl.py

--2025-11-29 21:55:59--  https://raw.githubusercontent.com/huggingface/diffusers/v0.29.2/examples/text_to_image/train_text_to_image_lora_sdxl.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55538 (54K) [text/plain]
Saving to: ‘train_text_to_image_lora_sdxl.py’

train_text_to_image 100%[===================>]  54.24K  --.-KB/s    in 0.01s   

2025-11-29 21:55:59 (5.19 MB/s) - ‘train_text_to_image_lora_sdxl.py’ saved [55538/55538]



In [26]:
!pip install diffusers==0.29.2 transformers accelerate peft==0.10.0 bitsandbytes safetensors datasets

In [27]:
import torch
print(torch.cuda.is_available())

True


In [28]:
import torch, gc
from diffusers import DiffusionPipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [29]:
from diffusers import StableDiffusionXLPipeline
import torch
from PIL import Image
import os, glob
from pathlib import Path

In [17]:
!pip install flask pyngrok diffusers transformers accelerate safetensors torch torchvision

In [30]:
%%writefile app.py
from flask import Flask, request, jsonify, send_from_directory
import torch
from diffusers import DiffusionPipeline

app = Flask(__name__)

print("Loading SDXL pipeline...")

base_model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

lora_path = "/content/drive/MyDrive/floorplan_lora/pytorch_lora_weights.safetensors"
pipe.load_lora_weights(lora_path)

print("Pipeline loaded with LoRA.")

@app.route("/", methods=["GET"])
def home():
    return "SDXL LoRA API is running!"

@app.route("/generate", methods=["POST"])
def generate():
    data = request.get_json()

    prompt = data.get("prompt")
    width = int(data.get("width", 1024))
    height = int(data.get("height", 1024))

    if not prompt:
        return jsonify({"error": "Prompt is required"}), 400

    # Run SDXL + LoRA
    image = pipe(
        prompt=prompt,
        width=width,
        height=height,
        num_inference_steps=30,
    ).images[0]

    filename = "generated.png"
    image.save(filename)

    # Make the file accessible via ngrok
    public_url = request.host_url + filename

    return jsonify({
        "message": "Image generated successfully",
        "url": public_url
    })

@app.route('/<path:filename>', methods=['GET'])
def serve_file(filename):
    return send_from_directory(".", filename)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


Overwriting app.py


In [31]:
!ngrok config add-authtoken 33ZR4wpWS0uzdGYxBLkhpVn7ei3_41WjbmuoktxPVDFHVfstw

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [32]:
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print(public_url)
!python app.py;

NgrokTunnel: "https://unrehearsed-airtightly-gerard.ngrok-free.dev" -> "http://localhost:5000"
2025-11-29 21:59:16.659660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764453556.691785   31116 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764453556.701631   31116 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764453556.726769   31116 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764453556.726804   31116 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more tha

Pipeline loaded with LoRA.
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Nov/2025 22:00:59] "POST / HTTP/1.1" 405 -
127.0.0.1 - - [29/Nov/2025 22:01:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 22:01:05] "POST / HTTP/1.1" 405 -
127.0.0.1 - - [29/Nov/2025 22:01:47] "POST / HTTP/1.1" 405 -
100% 30/30 [00:27<00:00,  1.08it/s]
127.0.0.1 - - [29/Nov/2025 22:12:48] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 22:13:27] "GET /generated.png HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 22:14:14] "GET /favicon.ico HTTP/1.1" 404 -
100% 30/30 [00:28<00:00,  1.04it/s]
127.0.0.1 - - [29/Nov/2025 22:15:07] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 22:18:08] "GET /generated.png HTTP/1.1" 200 -
100% 30/30 [00:28<00:00,  1.04it/s]
127.0.0.1 - - [29/Nov/2025 22:19:27] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [29/N

📌 Step 8 — Resume Training & Improve Model Quality

In this step, we continue training the SDXL LoRA model from an existing checkpoint (2000 steps).
This allows the model to keep learning while preserving what it already understood before.

We load the dataset again, unpack the saved project folder, and resume training with optimized settings such as:

Gradient accumulation → stabilizes training on small batch size

FP16 mixed precision → faster + less GPU memory

8-bit Adam optimizer → reduced memory usage

Regular checkpoint saving → ensures we can always resume training safely

This process helps refine the model and improve the quality of generated floorplans.

In [ ]:
!unzip /content/drive/MyDrive/floorplan_lora-20251126T133920Z-1-001.zip

Archive:  /content/drive/MyDrive/floorplan_lora-20251126T133920Z-1-001.zip
  inflating: floorplan_lora/checkpoint-1500/scaler.pt  
  inflating: floorplan_lora/checkpoint-1500/random_states_0.pkl  
  inflating: floorplan_lora/checkpoint-500/scaler.pt  
  inflating: floorplan_lora/checkpoint-500/random_states_0.pkl  
  inflating: floorplan_lora/checkpoint-500/scheduler.bin  
  inflating: floorplan_lora/checkpoint-1500/scheduler.bin  
  inflating: floorplan_lora/checkpoint-2000/random_states_0.pkl  
  inflating: floorplan_lora/checkpoint-1000/scaler.pt  
  inflating: floorplan_lora/checkpoint-1000/scheduler.bin  
  inflating: floorplan_lora/checkpoint-2000/scaler.pt  
  inflating: floorplan_lora/checkpoint-2000/scheduler.bin  
  inflating: floorplan_lora/checkpoint-1000/random_states_0.pkl  
  inflating: floorplan_lora/checkpoint-500/pytorch_lora_weights.safetensors  
  inflating: floorplan_lora/checkpoint-1500/pytorch_lora_weights.safetensors  
  inflating: floorplan_lora/pytorch_lora_we

In [ ]:
!accelerate launch train_text_to_image_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --train_data_dir="train_data" \
  --caption_column="text" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=8 \
  --learning_rate=5e-5 \
  --max_train_steps=6000 \
  --resume_from_checkpoint="checkpoint-2000" \
  --checkpointing_steps=500 \
  --output_dir="/content/drive/MyDrive/floorplan_lora" \
  --mixed_precision="fp16" \
  --use_8bit_adam


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `0`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/usr/bin/python3: can't open file '/content/train_text_to_image_lora_sdxl.py': [Errno 2] No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/accelerate_cli.py", line 50, in main
    args.func(args)
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/launch.py", line 1281, in launch_command
    simple_launcher(args)
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/launch.py", line 869, in simple_launcher
